使用开源模型ChatYuan-large-v1模型，在法律场景下进行评估，并横向对表法域通效果，测试：
- 智能问答
- 多轮咨询
- 信息抽取
- 内容生成

In [3]:
# 加载需要用到的模型
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

import time

st_time = time.time()
tokenizer = T5Tokenizer.from_pretrained("../model/language_model/ChatYuan-large-v1")
model = T5ForConditionalGeneration.from_pretrained("../model/language_model/ChatYuan-large-v1")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print("load model time: ", time.time() - st_time)

load model time:  11.246935606002808


In [4]:
# 基础的预处理和后处理
def preprocess(text):
    text = text.replace("\n", "\\n").replace("\t", "\\t")
    return text


def postprocess(text):
    return text.replace("\\n", "\n").replace("\\t", "\t")


def answer(text, sample=True, top_p=1, temperature=0.7):
    """sample：是否抽样。生成任务，可以设置为True;
    top_p：0-1之间，生成的内容越多样"""
    text = preprocess(text)
    encoding = tokenizer(text=[text], truncation=True, padding=True, max_length=768, return_tensors="pt").to(device)
    if not sample:
        out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_new_tokens=512,
                             num_beams=1, length_penalty=0.6)
    else:
        out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_new_tokens=512,
                             do_sample=True, top_p=top_p, temperature=temperature, no_repeat_ngram_size=3)
    out_text = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)
    return postprocess(out_text[0])

## 1、简单的问答

In [5]:
# 直接用来完成法律问答任务
question = "宅基地可以转让吗?"
input_text_1 = "问题："+question+"\n答案："
output_text_1 = answer(input_text_1, sample=True)
input_text_2 = "当事人："+question+"\n律师："
output_text_2 = answer(input_text_2, sample=True)
print("prompt1:\n",output_text_1)
print("prompt2:\n",output_text_2)

prompt1:
 宅基地是不能转让的。宅基地是指农村集体经济组织成员依法取得的宅基地使用权，除法律、法规另有规定外，不得转让。宅基证上的“宅基地”是指农村村民依法取得的一个特定土地使用权。宅地是农村集体经济组织的集体土地，它是由农村集体经济组织和村民依法以土地使用为目的而拥有的集体所有的建筑物和构筑物，是农村集体所有的合法财产。《中华人民共和国土地管理法》等规定：“农村村民一户只能拥有一处宅基地，其宅基地的面积不得超过省、自治区、直辖市规定的标准。农村村民出卖、出租住房后，再申请宅基地并取得宅基地使用的，不予批准”。
prompt2:
 宅基地不能转让


- 法域通回答结果

    首先，宅基地使用权的转让必须同时具备以下条件，缺一不可：（1）转让人拥有二处以上的农村住房（含宅基地）；（2）同一集体经济组织内部成员转让；（3）受让人没有住房和宅基地，符合宅基地使用权分配条件；（4）转让行为征得集体组织同意；（5）宅基地使用权不得单独转让，地随房一并转让。其次，宅基地使用权不得单独转让，有下列转让情况，会被认定为无效：（1）城镇居民购买；（2）法人或其他组织购买；（3）转让人未经集体组织批准；（4）向集体组织成员以外的人转让；（5）受让人已有住房，不符合宅基地分配条件。　因此，只有符合以上条件，宅基地才可以进行转让，如果在转让宅基地时发生纠纷，可以根据以下方法解决：（1）协商解决公民之间发生的宅基地纠纷，应当先通过协商的方式加以解决。（2）行政解决个人之间，个人与单位之间关于侵犯土地的所有权或者使用权争议，可以向乡级人民政府或者县级以上人民政府反映处理。（3）司法解决公民之间就土地的使用权和所有权发生的纠纷，可以先经过有关行政机关的处理，对于处理决定不服，才可以向人民法院提起诉讼。否则，人民法院不予受理。但对于侵犯土地的所有权或者使用权的，被侵权人可以不经行政机关的处理，而直接向人民法院起诉。
- chatgpt回答结果

    是的，宅基地可以转让。但是需要遵守相关法律法规，如房屋买卖合同等，并经过合法程序，如交易所需的证明文件和登记等。建议咨询相关专业人员以确保合法合规。

In [7]:
question = "我被邻居家的狗咬了怎么办?"
input_text_1 = "问题："+question+"\n答案："
output_text_1 = answer(input_text_1, sample=True)
input_text_2 = "当事人："+question+"\n律师："
output_text_2 = answer(input_text_2, sample=True)
print("prompt1:\n",output_text_1)
print("prompt2:\n",output_text_2)

prompt1:
 你好，协商不成，可以向法院起诉，要求赔偿。
prompt2:
 您好，可以要求邻居赔偿的


- 法域通回答结果

    孕妇被狗咬了的处理方法：1、应即刻接种狂犬疫苗。通常情况下，孕前期及孕期按照常规使用狂犬病疫苗，对孕妇的健康和胎儿的生长发育都是安全的。2、被他人的狗咬伤，还可以追究侵权责任。根据《民法典》规定，饲养的动物造成他人损害的，动物饲养人或者管理人应当承担侵权责任，但能够证明损害是因被侵权人故意或者重大过失造成的，可以不承担或者减轻责任。法律依据：《民法典》第一千二百四十五条饲养的动物造成他人损害的，动物饲养人或者管理人应当承担侵权责任；但是，能够证明损害是因被侵权人故意或者重大过失造成的，可以不承担或者减轻责任。
- chatgpt回答结果

    如果您被狗咬了，建议您立即就医，以预防狂犬病等感染。请确保保存证据，如伤口照片，看病证明等。如果您认为狗主人应对此事负责，可以考虑联系当地法律机构寻求帮助。

## 2、多轮问答

In [9]:
input_text = ["我被邻居家的狗咬了怎么办?"]

answer_text = ["你好，协商不成，可以向法院起诉，要求赔偿。"]

context = "\n".join([f"当事人：{input_text[i]}\n律师：{answer_text[i]}" for i in range(len(input_text))])
# print(context)
#
input_text = "我要起诉对方狗主人，我应该提出哪些诉求？"
input_text = context + "\n当事人：" + input_text + "\n律师："
output_text = answer(input_text)

print(output_text)

你好，建议携带相关证据直接到法院起诉。


- 法域通回答结果

    目前不支持
- chatgpt回答结果
    如果您要起诉对方狗主人，您可以提出以下诉求：

    1.赔偿医疗费用：要求对方赔偿您因狗咬伤所造成的医疗费用。
    2.赔偿精神损害费：要求对方赔偿您因狗咬伤所造成的精神损害费用。
    3.赔偿工作损失：如果您因狗咬伤而导致工作中断，可以要求对方赔偿您的工作损失。
    4.对狗主人的教育和培训要求：要求对方对狗进行教育和培训，以预防未来再次发生类似事件。
    请注意，不同地区的法律法规和判例可能有所不同，因此建议您请教专业律师以了解更多相关信息。

## 3、信息抽取

In [16]:
content = """借条
本人杨宇（身份证号码：152222197210314151 ），因资金周转需要，于 2022 年 1月 1 日向李凤英（身份证号码：610200198406204822），借款3000元（大写：元整）。借期 6个月 ，年利率 12%（百分之十二），2022年6月30日到期时本息一并还清。如到期未还清，杨宇愿按年利率12% （百分之十二）计付逾期利息，并同意承担李凤英 通过诉讼等方式追讨借款时的律师费、保全担保费等相关费用。借款人确认浙江省杭州市滨江拉萨路W座地址作为送达催款函，法院送达文书诉讼文书的地址，若借款人未及时书面告知出借人变更后的地址，导致相关文书及诉讼文书未能实际被接收的、邮寄送达的，相关文书及诉讼文书退回之日即视为送达之日。借款人：杨宇2022年1月1日"""
input_text = "信息抽取：\n" + content + "\n问题：标题，借款人，出借人，借款期限，借款金额，借款方式，借款利率\n答案："
output_text = answer(input_text)
print(output_text)

信息抽取：
借条
本人杨宇（身份证号码：152222197210314151 ），因资金周转需要，于 2022 年 1月 1 日向李凤英（身份证号码：610200198406204822），借款3000元（大写：元整）。借期 6个月 ，年利率 12%（百分之十二），2022年6月30日到期时本息一并还清。如到期未还清，杨宇愿按年利率12% （百分之十二）计付逾期利息，并同意承担李凤英 通过诉讼等方式追讨借款时的律师费、保全担保费等相关费用。借款人确认浙江省杭州市滨江拉萨路W座地址作为送达催款函，法院送达文书诉讼文书的地址，若借款人未及时书面告知出借人变更后的地址，导致相关文书及诉讼文书未能实际被接收的、邮寄送达的，相关文书及诉讼文书退回之日即视为送达之日。借款人：杨宇2022年1月1日
问题：标题，借款人，出借人，借款期限，借款金额，借款方式，借款利率
答案：
385
借款人，出借人，借款期限


- 法域通回答结果

    标题：借条
    借款人：杨宇
    出借人：李凤英
    借款期限：6个月
    借款金额：借款3000元（大写：元整）
    借款方式：/
    借款利率：年利率 12%（百分之十二
- chatgpt回答结果

    标题: 借款协议
    借款人: 杨宇
    出借人: 李凤英
    借款期限: 6个月
    借款金额: 3000元
    借款方式: 现金借款
    借款利率: 12%

## 4、内容生成

In [17]:
input_text = """根据标题生成文章：借条\n答案:"""
output_text = answer(input_text)
print(output_text)

一、借条的签章及内容1、本借条为借款人签字，借款人为姓名，年月日，身份证号码，身份证复印件。2、本借据在借据上注明“借款”字样，“借款用途”、“期限”、“利息”等事项，并附借款人身份证明、借款借据复印件。3、借款人签名后捺印。二、本借书的起止日期为:年月日至日，即借据签订后开始至还款日止。三、本合同在双方签订合同前，经双方协商一致，由被告向原告出具借条并交付现金。合同签订后，被告向被告交付现金200万元。四、本条所称借款，是指原告向被告借款，且借款金额在人民币元以内，并注明“借据”字样。五、本借款合同的签订，目的在于保护债权人的利益，维护双方当事人的合法权益，保证诉讼顺利进行。


- 法域通回答结果

    暂不支持
- chatgpt回答结果

    借条
    日期：[日期]
    借出方：[借出方姓名]
    借入方：[借入方姓名]
    借出金额：[借出金额]
    还款日期：[还款日期]
    利率：[利率]

    借入方同意在[还款日期]前归还借出方[借出金额]元人民币。如果逾期未还，同意按照[利率]的利率支付逾期利息。

    签字：
    借出方： [借出方签字]
    借入方： [借入方签字]

    请注意，这仅是一份模板，并不适用于所有情况，在使用前请确保符合当地法律规定并经过相关法律意见。


In [18]:
input_text = """根据标题生成文章：离婚起诉书\n答案:"""
output_text = answer(input_text)
print(output_text)

原告:张先生,男,汉族,1962年7月生,户籍:山东省济南市历下区,住址:山东省济南历下区人民法院。被告:张女士,女,汉族,1963年12月生,籍贯:山东省青岛市,户籍地址:山东省青岛莱西市,住所:山东省莱西市中级人民法院。原告诉称:因感情不和,于2015年6月30日原告与被告协议离婚。离婚后原告与子女共同生活,孩子归原告抚养,其抚养费由被告承担。原告与原被告因孩子抚养问题多次协商未果,故诉至法院,要求判令被告支付抚养费,并要求被告支付精神抚慰金。


- 法域通回答结果

    暂不支持
- chatgpt回答结果
    以下是一个离婚起诉书的模板，请注意，这仅供您参考，不能作为法律文件的替代。请在使用前咨询专业的法律顾问：

    [Your Name]
    [Address]
    [City, State ZIP Code]
    [Phone Number]
    [Email Address]

    [Date]

    The Honorable [Judge's Name]
    [Court's Name]
    [Address]
    [City, State ZIP Code]

    Re: Complaint for Divorce

    [Your Spouse's Name]
    [Address]
    [City, State ZIP Code]

    Dear Judge [Judge's Name],

    I, [Your Name], respectfully submit this complaint for divorce against my spouse, [Spouse's Name]. Our marriage was solemnized on [Date of Marriage], and we have been living separate and apart since [Date of Separation].

    The grounds for divorce are as follows:
    [Insert grounds for divorce, such as irreconcilable differences, cruelty, adultery, abandonment, etc.]

    In addition to the grounds for divorce, I request the court to consider the following issues:
    [Insert any other relevant issues, such as property division, spousal support, child custody, etc.]

    I respectfully request that this Honorable Court grant me a divorce and make such further orders as are just and equitable.

    Thank you for your time and consideration.

    Sincerely,

    [Your Signature]
    [Your Name (Typed)]

## 5. 分析

- 法域通回答结果

    较为中规中矩，但是对于一些特殊的问题，如“借条”中的“借款利率”等，法域通回答结果需要进行定制化的训练，这是因为法域通的模型是基于大量的法律文书进行训练的，而借条中的“借款利率”这一项并不是法律文书中的常见项，因此模型并没有学习到这一项，因此在回答时无法给出答案。

- chatgpt回答结果

    可以灵活的生成各种文书，但是对于一些具体的法律问题，chatgpt给出的结果未必是正确的，而且出现了一些英文结果，这是因为chatgpt是基于大量的英文文本进行训练的，因此在生成中文文书时，会出现一些英文结果。

- ChatYuan-large-v1

    ChatYuan-large-v1是基于大量的中文文本进行训练的，因此在生成中文文书时，不会出现英文结果，但是对于一些具体的法律问题，ChatYuan-large-v1目前给出的结果还不是太理想。

ps：以上分析主要由Github Coplit工具生成，Coplit是一个基于GPT-3的文本生成工具，可以生成各种文本，包括代码、文档、文案等，详情请见：https://coplit.ai/。